# 11: Streaming Pipeline Demo - Pulsar Integration

**Date:** 2026-02-06  
**Author:** AdaL  
**Status:** Active

## Overview

This notebook demonstrates the **event-sourced streaming architecture** using Apache Pulsar for real-time data ingestion into JanusGraph and OpenSearch.

---

## Architecture

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│ Data Generators │────▶│  Apache Pulsar  │────▶│   Consumers     │
│                 │     │  (Event Bus)    │     │                 │
│  • Persons      │     │                 │     │  ┌───────────┐  │
│  • Accounts     │     │  Topics:        │     │  │ GraphCons │──┼──▶ JanusGraph
│  • Transactions │     │  • persons      │     │  └───────────┘  │
│  • Companies    │     │  • accounts     │     │                 │
│  • etc.         │     │  • transactions │     │  ┌───────────┐  │
└─────────────────┘     │  • companies    │     │  │VectorCons │──┼──▶ OpenSearch
                        │  • dlq          │     │  └───────────┘  │
                        └─────────────────┘     └─────────────────┘
```

### Key Benefits

| Feature | Description |
|---------|-------------|
| **ID Consistency** | Same UUID across Pulsar, JanusGraph, OpenSearch |
| **Dual-Leg Ingestion** | Parallel writes to graph and vector stores |
| **Fault Tolerance** | DLQ for failed messages with retry logic |
| **Real-Time** | ~115ms latency from event to queryable data |

---

## Prerequisites

```bash
# Start all services including Pulsar
cd config/compose && bash ../../scripts/deployment/deploy_full_stack.sh
```

## Section 1: Setup and Imports

In [ ]:
# Fix event loop for Jupyter
import nest_asyncio
nest_asyncio.apply()

# Standard imports
import sys
from pathlib import Path
import os
from datetime import datetime
import json

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Streaming imports
from banking.streaming import (
    EntityEvent,
    EntityProducer,
    MockEntityProducer,
    create_person_event,
    create_account_event,
    create_transaction_event,
    StreamingOrchestrator,
    StreamingConfig,
    get_metrics_output,
)

DEMO_SEED = int(os.getenv("DEMO_SEED", "42"))
DEMO_FORCE_MOCK_PULSAR = os.getenv("DEMO_FORCE_MOCK_PULSAR", "").lower() in ("1", "true", "yes", "on")
print("✅ Imports successful")
print(f"   Project root: {project_root}")
print(f"   DEMO seed: {DEMO_SEED}")

## Section 2: Check Pulsar Availability

In [ ]:
def check_pulsar():
    """Check if Pulsar is available."""
    try:
        import pulsar
        client = pulsar.Client('pulsar://localhost:6650', operation_timeout_seconds=5)
        client.close()
        return True
    except Exception as e:
        print(f"⚠️ Pulsar not available: {e}")
        return False

PULSAR_AVAILABLE = check_pulsar()

if PULSAR_AVAILABLE:
    print("✅ Pulsar is available at pulsar://localhost:6650")
else:
    print("⚠️ Pulsar not available - will use MockEntityProducer for demos")

## Section 3: Understanding EntityEvent Schema

The `EntityEvent` is the core data structure for streaming. It captures:
- **entity_id**: Unique identifier (used as partition key)
- **entity_type**: Type of entity (person, account, transaction, etc.)
- **event_type**: What happened (create, update, delete)
- **payload**: The actual entity data
- **metadata**: Optional metadata (source, version, etc.)

In [ ]:
# Create a sample EntityEvent manually
sample_event = EntityEvent(
    entity_id="person-demo-001",
    entity_type="person",
    event_type="create",
    timestamp=datetime.utcnow(),
    source="NotebookDemo",
    version="1.0",
    payload={
        "id": "person-demo-001",
        "first_name": "Alice",
        "last_name": "Johnson",
        "email": "alice.johnson@example.com",
        "risk_score": 0.2,
        "country": "United States"
    },
    text_for_embedding="Alice Johnson, customer from United States, email alice.johnson@example.com"
)

print("📦 Sample EntityEvent:")
print(f"   Entity ID: {sample_event.entity_id}")
print(f"   Type: {sample_event.entity_type}")
print(f"   Event: {sample_event.event_type}")
print(f"   Topic: {sample_event.get_topic()}")
print()
print("📄 JSON representation:")
print(json.dumps(sample_event.to_dict(), indent=2, default=str))

## Section 4: Using Factory Functions

Factory functions make it easy to create events for common entity types.

In [ ]:
# Create events using factory functions
person_event = create_person_event(
    person_id="person-factory-001",
    name="Bob Smith",
    payload={
        "id": "person-factory-001",
        "first_name": "Bob",
        "last_name": "Smith",
        "email": "bob.smith@example.com"
    },
    source="NotebookDemo"
)

account_event = create_account_event(
    account_id="account-factory-001",
    payload={
        "account_id": "account-factory-001",
        "account_type": "checking",
        "balance": 5000.00,
        "currency": "USD",
        "owner_id": "person-factory-001"
    },
    source="NotebookDemo"
)

transaction_event = create_transaction_event(
    transaction_id="txn-factory-001",
    payload={
        "transaction_id": "txn-factory-001",
        "amount": 250.00,
        "transaction_id": "txn-factory-001",
        "amount": 250.00,
        "currency": "USD",
        "from_account": "account-factory-001",
        "to_account": "account-factory-002",
        "type": "transfer"
    },
    source="NotebookDemo"
)

print("✅ Events created using factory functions:")
print(f"   Person: {person_event.entity_id} → {person_event.get_topic()}")
print(f"   Account: {account_event.entity_id} → {account_event.get_topic()}")
print(f"   Transaction: {transaction_event.entity_id} → {transaction_event.get_topic()}")

## Section 5: Publishing Events to Pulsar

The `EntityProducer` handles publishing events to the appropriate Pulsar topics.

In [ ]:
# Choose producer based on Pulsar availability
if PULSAR_AVAILABLE:
    producer = EntityProducer(pulsar_url="pulsar://localhost:6650")
    print("📡 Using real Pulsar producer")
else:
    producer = MockEntityProducer()
    print("🎭 Using mock producer (Pulsar not available)")

# Publish events
events_to_publish = [person_event, account_event, transaction_event]

for event in events_to_publish:
    producer.send(event)
    print(f"   📤 Published: {event.entity_type}/{event.entity_id}")

# Flush to ensure delivery
producer.flush()

# Get stats
stats = producer.get_stats()
print("\n📊 Producer Stats:")
print(f"   Total sent: {stats.get('total_sent', 0)}")
print(f"   By type: {stats.get('by_entity_type', {})}")

## Section 6: StreamingOrchestrator - End-to-End Pipeline

The `StreamingOrchestrator` extends the data generators to automatically publish events to Pulsar.

In [ ]:
import tempfile
import shutil

# Create a streaming configuration
output_root = Path(os.getenv("DEMO_FIXED_OUTPUT_DIR", tempfile.mkdtemp()))
config = StreamingConfig(
    seed=DEMO_SEED,  # For reproducibility
    person_count=5,
    company_count=2,
    account_count=10,
    transaction_count=20,
    communication_count=5,
    trade_count=0,
    travel_count=0,
    document_count=0,
    use_mock_producer=DEMO_FORCE_MOCK_PULSAR or not PULSAR_AVAILABLE,  # Use real Pulsar if available
    pulsar_url="pulsar://localhost:6650",
    output_dir=output_root
)

print("⚙️ StreamingConfig:")
print(f"   Persons: {config.person_count}")
print(f"   Companies: {config.company_count}")
print(f"   Accounts: {config.account_count}")
print(f"   Transactions: {config.transaction_count}")
print(f"   Communications: {config.communication_count}")
print(f"   Mock Producer: {config.use_mock_producer}")

In [ ]:
# Run the streaming orchestrator
print("🚀 Starting StreamingOrchestrator...\n")

try:
    with StreamingOrchestrator(config) as orchestrator:
        stats = orchestrator.generate_all()
        
        print("\n📊 Generation Results:")
        print(f"   Persons generated: {stats.persons_generated}")
        print(f"   Companies generated: {stats.companies_generated}")
        print(f"   Accounts generated: {stats.accounts_generated}")
        print(f"   Transactions generated: {stats.transactions_generated}")
        print(f"   Communications generated: {stats.communications_generated}")
        print()
        print("📤 Streaming Results:")
        print(f"   Events published: {stats.events_published}")
        print(f"   Events failed: {stats.events_failed}")
        print(f"   Events by type: {stats.events_by_type}")
finally:
    # Cleanup temp directory
    shutil.rmtree(config.output_dir, ignore_errors=True)

In [ ]:
# Plot streaming event counts by type

try:
    import os
    os.environ.setdefault("MPLBACKEND", "Agg")
    import matplotlib.pyplot as plt

    event_counts = getattr(stats, "events_by_type", {
        "person": getattr(stats, "persons_generated", 0),
        "company": getattr(stats, "companies_generated", 0),
        "account": getattr(stats, "accounts_generated", 0),
        "transaction": getattr(stats, "transactions_generated", 0),
        "communication": getattr(stats, "communications_generated", 0),
    })

    if not event_counts:
        print("⚠️ No event counts available to plot.")
    else:
        labels = list(event_counts.keys())
        values = [event_counts[label] for label in labels]

        plt.figure(figsize=(9, 4))
        plt.bar(labels, values, color=["#4f46e5", "#2563eb", "#0ea5e9", "#14b8a6", "#10b981"])
        plt.title("Streaming Event Counts by Entity Type")
        plt.xlabel("Entity Type")
        plt.ylabel("Events Published")
        plt.grid(axis="y", alpha=0.3)
        plt.tight_layout()
        plt.show()

        print("📊 Plot generated successfully.")
except Exception as exc:
    print(f"⚠️ Plot generation skipped: {exc}")

## Section 7: ID Consistency Guarantee

The architecture guarantees that the **same UUID** is used across all systems:

```
EntityEvent.entity_id  ─┬─▶ Pulsar partition_key
                        ├─▶ JanusGraph vertex.entity_id property
                        └─▶ OpenSearch document._id
```

This enables cross-system joins, deduplication, and consistent audit trails.

In [ ]:
# Demonstrate ID consistency
demo_event = create_person_event(
    person_id="consistency-demo-uuid-12345",
    name="Consistency Demo",
    payload={"id": "consistency-demo-uuid-12345", "name": "Demo"},
    source="ConsistencyDemo"
)

# Get Pulsar message format
pulsar_msg = demo_event.to_pulsar_message()

print("🔑 ID Consistency Demonstration:")
print(f"\n   EntityEvent.entity_id:     {demo_event.entity_id}")
print(f"   Pulsar partition_key:      {pulsar_msg['partition_key']}")
print(f"   Payload['id']:             {demo_event.payload.get('id')}")
print()
print("   ✅ All IDs match - guaranteed consistency across systems!")
print()
print("   In production:")
print(f"   → JanusGraph: g.V().has('person', 'entity_id', '{demo_event.entity_id}')")
print(f"   → OpenSearch: GET /persons/_doc/{demo_event.entity_id}")

## Section 8: Prometheus Metrics

The streaming module exports Prometheus metrics for monitoring.

In [ ]:
# Get Prometheus metrics
metrics_output = get_metrics_output()

# Decode bytes to string
metrics_text = metrics_output.decode('utf-8') if isinstance(metrics_output, bytes) else metrics_output

print("📈 Prometheus Metrics (sample):")
print("="*60)

# Show a subset of interesting metrics
for line in metrics_text.split('\n'):
    if line.startswith('streaming_') and not line.startswith('#'):
        print(line)
        
print("="*60)
print("\n💡 These metrics can be scraped by Prometheus for monitoring dashboards")

## Section 9: Cleanup

In [ ]:
# Cleanup
print("🧹 Cleaning up...")

if 'producer' in dir() and producer:
    producer.close()
    print("   ✅ Producer closed")

print()
print("="*70)
print("✅ STREAMING PIPELINE DEMO COMPLETE")
print("="*70)

## Summary

This notebook demonstrated:

1. **EntityEvent Schema** - The core data structure for streaming
2. **Factory Functions** - Easy event creation for common entity types
3. **EntityProducer** - Publishing events to Pulsar topics
4. **StreamingOrchestrator** - End-to-end pipeline from generation to publishing
5. **ID Consistency** - Same UUID across all systems
6. **Prometheus Metrics** - Monitoring and observability

### Related Documentation

- [Streaming Module README](../../banking/streaming/README.md)
- [Streaming Architecture](../../docs/architecture/streaming-architecture.md)
- [Event-Sourced Ingestion Architecture](../../docs/architecture/EVENT_SOURCED_INGESTION_ARCHITECTURE.md)